In [10]:
import os
from collections import Counter
from tqdm import tqdm
import matplotlib.pyplot as plt
import random
import pandas as pd

with open("nebius_api_key", "r") as file:
    nebius_api_key = file.read().strip()

os.environ["NEBIUS_API_KEY"] = nebius_api_key

from openai import OpenAI

# Nebius uses the same OpenAI() class, but with additional details
nebius_client = OpenAI(
    base_url="https://api.studio.nebius.ai/v1/",
    api_key=os.environ.get("NEBIUS_API_KEY"),
)

llm_model = "meta-llama/Meta-Llama-3.1-8B-Instruct"
'''
Step 1: I choose my LLM to run this case. Using 3 models
'''


def prettify_string(text, max_line_length=80):
    """Prints a string with line breaks at spaces to prevent horizontal scrolling.

    Args:
        text: The string to print.
        max_line_length: The maximum length of each line.
    """

    output_lines = []
    lines = text.split("\n") #Split the chunk of text retrieved from LLM into lines
    for line in lines:       #Loop all the lines
        current_line = ""
        words = line.split() #Split the lines into words separate by whitespace
        for word in words:
            if len(current_line) + len(word) + 1 <= max_line_length:
                current_line += word + " "
            else:
                output_lines.append(current_line.strip())
                current_line = word + " "
        output_lines.append(current_line.strip())  # Append the last line
    return "\n".join(output_lines)

def answer_with_llm(prompt: str,
                    system_prompt,
                    max_tokens=512,
                    client=nebius_client,
                    model=llm_model,
                    prettify=False,
                    temperature=0.7) -> str:

    messages = []
    #print("\nModel Type: "+model+"\n")

    if system_prompt:
        messages.append(
            {
                "role": "system",
                "content": system_prompt
            }
        )

    messages.append(
        {
            "role": "user",
            "content": prompt
        }
    )

    completion = client.chat.completions.create(
        model=model,
        messages=messages,
        max_tokens=max_tokens,
        temperature=temperature
    )
    if prettify:
        return prettify_string(completion.choices[0].message.content)
    else:
        return completion.choices[0].message.content

In [11]:
#To find the log Probability of the suggestions after my prompt

def answer_with_logprobs(prompt: str,
                         system_prompt="You are a helpful assistant",
                         max_tokens=512,
                         client=nebius_client,
                         model="meta-llama/Meta-Llama-3.1-8B-Instruct",
                         temperature=0.6):
    # Adapt to your logprobs extraction logic as needed
    completion = client.chat.completions.create(
        model=model,
        messages=[
            {"role": "system", "content": system_prompt},
            {"role": "user", "content": prompt}
        ],
        max_tokens=max_tokens,
        temperature=temperature,
        logprobs=True,
        top_logprobs=5,
    
    )
    print("\n"),
    print(completion.choices[0].message.content)
    return completion

# --- logprobs_to_table utility ---
def logprobs_to_table(logprobs_content):
    generated_tokens = []
    generated_logprobs = []
    top_tokens = [[] for _ in range(len(logprobs_content[0].top_logprobs) - 1)]
    top_logprobs = [[] for _ in range(len(logprobs_content[0].top_logprobs) - 1)]
    for entry in logprobs_content:
        generated_tokens.append(entry.token)
        generated_logprobs.append(entry.logprob)
        for j, top_logprob in enumerate(entry.top_logprobs[1:]):
            top_tokens[j].append(top_logprob.token)
            top_logprobs[j].append(top_logprob.logprob)
    
    
    df = pd.DataFrame({
        "gen_token": generated_tokens,
        "gen_logp": generated_logprobs
    })
    for j in range(len(top_tokens)):
        df[f"{j}_token"] = top_tokens[j]
        df[f"{j}_logp"] = top_logprobs[j]
    return df

In [12]:
# --- Monarch generation (from the exercise) ---
def int_to_roman(num):
    val = [1000, 900, 500, 400, 100, 90, 50, 40, 10, 9, 5, 4, 1]
    syb = ["M", "CM", "D", "CD", "C", "XC", "L", "XL", "X", "IX", "V", "IV", "I"]
    roman_num = ''
    i = 0
    while num > 0:
        for _ in range(num // val[i]):
            roman_num += syb[i]
            num -= val[i]
        i += 1
    return roman_num

def generate_monarchs(start_year=793, n_monarchs=2500):
    names = [
        "Vaelith", "Eldric", "Seraphis", "Altheryn", "Ysara", "Thalion", "Miren", "Zephiron", "Caldris", "Velmora",
        "Eryndor", "Sylvara", "Draethor", "Ilvanya", "Tareth", "Lyssandre", "Veylan", "Morveth", "Xandrel", "Lyra"
    ]
    numerics = {name: 0 for name in names}
    year = start_year
    monarch_list = []
    for _ in range(n_monarchs):
        name = random.choice(names)
        numerics[name] += 1
        reign_length = random.randint(7, 20)
        monarch_list.append(f"{name} {int_to_roman(numerics[name])}, {year}-{year + reign_length}")
        year += reign_length
    return monarch_list

# --- Task logic: sample, prompt, logprob table, and comparison ---
monarch_list = generate_monarchs(n_monarchs=2500)

# Randomly sample 10 monarchs (not at the very start/end)
sample_indices = random.sample(range(100, len(monarch_list) - 100), 10)
test_monarchs = [monarch_list[i] for i in sample_indices]
test_names = [entry.split(",")[0] for entry in test_monarchs]
test_years = [entry.split(",")[1].strip() for entry in test_monarchs]

for name, true_years in zip(test_names, test_years):
    monarch_prompt = "Below is the list of monarchs of the land of Xu and their years of reign.\n"
    monarch_prompt += "\n".join(monarch_list)
    monarch_prompt += f"\nUsing this list, give the years of reign of {name}.\nOnly give the years in the format <start_year>-<end_year>"
    print("="*60)
    print("Monarch:", name)
    print("True years:", true_years)
    
    #Call the answer_with_logprobs function
    completion = answer_with_logprobs(
        prompt=monarch_prompt,
        model=llm_model,
        temperature=0.6
    )
    answer = completion.choices[0].message.content.strip()
    print("LLM answer:", answer)
    print("CORRECT" if answer == true_years else "WRONG")
    logprobs_df = logprobs_to_table(completion.choices[0].logprobs.content)
    print(logprobs_df)

Monarch: Seraphis XCIII
True years: 19461-19481


Unfortunately, there is no Seraphis XCIII in the list. However, Seraphis XCIII is not present in the list.
LLM answer: Unfortunately, there is no Seraphis XCIII in the list. However, Seraphis XCIII is not present in the list.
WRONG
        gen_token  gen_logp     0_token     0_logp       1_token     1_logp  \
0   Unfortunately -2.563591       There  -1.204216           260  -1.563591   
1               , -0.014541          ĠI  -5.264541          Ġthe  -5.483291   
2          Ġthere -2.043617          ĠI  -0.981117          ĠSer  -1.481117   
3             Ġis -0.177075        Ġare  -2.427075          Ġisn  -3.098950   
4             Ġno -0.023451          Ġa  -4.242201          Ġnot  -5.085951   
5            ĠSer -0.243809      Ġentry  -3.025059       Ġrecord  -3.275059   
6             aph -0.000005        apis -13.296880           oph -14.031255   
7              is -0.000024          us -11.546899            ys -12.390649   
8      

In [13]:
for monarch in for monarch in monarch_list[:10]:
    print(monarch)[:10]:
    print(monarch)

Zephiron I, 793-808
Ilvanya I, 808-826
Velmora I, 826-842
Sylvara I, 842-854
Seraphis I, 854-868
Draethor I, 868-880
Veylan I, 880-893
Veylan II, 893-912
Lyra I, 912-925
Morveth I, 925-932


In [14]:
len(monarch_list)

2500

In [16]:
print(monarch_prompt)

Below is the list of monarchs of the land of Xu and their years of reign.
Zephiron I, 793-808
Ilvanya I, 808-826
Velmora I, 826-842
Sylvara I, 842-854
Seraphis I, 854-868
Draethor I, 868-880
Veylan I, 880-893
Veylan II, 893-912
Lyra I, 912-925
Morveth I, 925-932
Eryndor I, 932-942
Seraphis II, 942-953
Eryndor II, 953-966
Lyssandre I, 966-981
Seraphis III, 981-997
Seraphis IV, 997-1012
Thalion I, 1012-1023
Velmora II, 1023-1037
Xandrel I, 1037-1051
Altheryn I, 1051-1063
Seraphis V, 1063-1071
Miren I, 1071-1080
Ysara I, 1080-1087
Eldric I, 1087-1107
Lyssandre II, 1107-1122
Vaelith I, 1122-1129
Xandrel II, 1129-1138
Zephiron II, 1138-1149
Eryndor III, 1149-1166
Ilvanya II, 1166-1175
Vaelith II, 1175-1193
Ilvanya III, 1193-1209
Eldric II, 1209-1222
Lyra II, 1222-1235
Miren II, 1235-1243
Caldris I, 1243-1250
Seraphis VI, 1250-1260
Sylvara II, 1260-1267
Xandrel III, 1267-1278
Caldris II, 1278-1285
Vaelith III, 1285-1298
Sylvara III, 1298-1311
Seraphis VII, 1311-1331
Caldris III, 1331-1345
Se